In [1]:
import pandas as pd
import ujson as js
import json
from multiprocessing.dummy import Pool
from datetime import datetime as dt

from ApiHandler import ApiHandler

#from utils.email import send_email

In [2]:
class ApiRegions(ApiHandler):
    settings = {
        **ApiHandler.settings,
        'id_params': {
            'regions': {
                'id_col': 'region_id',
                'name_col': 'region_name',
                'sub_keys': ['constellations']
            },
            'constellations': {
                'id_col': 'constellation_id',
                'name_col': 'constellation_name',
                'sub_keys': ['systems']
            },
            'systems': {
                'id_col': 'system_id',
                'name_col': 'system_name',
                'sub_keys': ['stargates', 'planets', 'stations']
            },
            'planets': {
                'id_col': 'planet_id',
                'name_col': 'planet_name',
                'sub_keys': []
            },
            'moons': {
                'id_col': 'moon_id',
                'name_col': 'moon_name',
                'sub_keys': []
            },
            'asteroid_belts': {
                'id_col': 'asteroid_belt_id',
                'name_col': 'asteroid_belt_name',
                'sub_keys': []
            },
            'stargates': {
                'id_col': 'stargate_id',
                'name_col': 'stargate_name',
                'sub_keys': []
            },
            'stations': {
                'id_col': 'station_id',
                'name_col': 'station_name',
                'sub_keys': []
            }
        }
    }
    mappings = {
        'regions': {
            'region_name': 'name',
            'region_desc': 'description'
        },
        'constellations': {
            'constellation_name': 'name',
            'region_id': 'region_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'systems': {
            'system_name': 'name',
            'constellation_id': 'constellation_id',
            'security_class': 'security_class',
            'security_status': 'security_status',
            'star_id': 'star_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'planets': {
            'planet_name': 'name',
            'system_id': 'system_id',
            'type_id': 'type_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'moons': {
            'moon_name': 'name',
            'system_id': 'system_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'asteroid_belts': {
            'asteroid_belt_name': 'name',
            'system_id': 'system_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'stargates': {
            'system_id': 'system_id',
            'destination_stargate_id': 'destination.stargate_id',
            'destination_system_id': 'destination.system_id',
            'stargate_name': 'name',
            'type_id': 'type_id',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        },
        'stations': {
            'station_name': 'name',
            'system_id': 'system_id',
            'type_id': 'type_id',
            'max_dockable_ship_volume': 'max_dockable_ship_volume',
            'office_rental_cost': 'office_rental_cost',
            'owner': 'owner',
            'race_id': 'race_id',
            'reprocessing_efficiency': 'reprocessing_efficiency',
            'reprocessing_stations_take': 'reprocessing_stations_take',
            'pos_x': 'position.x',
            'pos_y': 'position.y',
            'pos_z': 'position.z'
        }
    }
    url = {
        **ApiHandler.url,
        'data': 'https://esi.evetech.net/latest/universe/{pull_name}/',
        'id_data': 'https://esi.evetech.net/latest/universe/{pull_name}/{id_val}/'
    }
    sql = {
        **ApiHandler.sql,
        'delete': 'DELETE FROM {table} WHERE {id_col} NOT IN ({id_vals});',
        'current_ids': 'SELECT {id_col} FROM {table} WHERE {name_col} IS NOT NULL;'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': {
            'regions': 'MapRegions',
            'constellations': 'MapConstellations',
            'systems': 'MapSystems',
            'planets': 'MapPlanets',
            'moons': 'MapMoons',
            'asteroid_belts': 'MapBelts',
            'stargates': 'MapStargates',
            'stations': 'MapStations'
        }
    }
    buffers = {}
    active_ids = {}
    name = 'ESI Map APIs'
    auth = False
    delete = True
    timestamp = None
    pull_sequence = ['regions', 'constellations', 'systems', 'planets', 'asteroid_belts', 'moons', 'stargates', 'stations']
    
    def __init__(self, verbose = False, thread_max=4):
        super().__init__(verbose=verbose)
        self.thread_max = thread_max
        
    def run_process(self):
        try:
            self.active_pull = self.pull_sequence[0]
            raw_data = self.get_raw_data()
            self.buffers[self.active_pull] = self.build_data(raw_data)
            for pull_name in self.pull_sequence[1:]:
                self.active_pull = pull_name
                raw_data = self.buffers[self.active_pull]
                self.buffers[self.active_pull] = self.build_data(raw_data)
            self.insert_data(self.buffers)
        except Exception as e:
            send_email('fail', {
                'process': self.name,
                'time': dt.now().isoformat(' '),
                'note': e
            })
        
    def get_raw_data(self):
        if self.verbose: self._verbose('get_raw_data', 'Getting raw data...')
        
        if self.auth:
            self.headers['Authorization'] = 'Bearer {token}'.format(**{
                'token': self.auth_token
            })
            
        data_conn = self.url_get(
            self.url['data'].format(**self.url_params, pull_name=self.active_pull),
            self.headers,
            self.params
        )
        
        if self.timestamp is None:
            self.timestamp = dt.strptime(
                data_conn.headers['Last-Modified'],
                '%a, %d %b %Y %H:%M:%S %Z'
            ).strftime('%Y-%m-%d %H:%M:%S')

        if self.verbose: self._verbose('get_raw_data', 'Raw data acquired.')
        
        raw_data = js.loads(data_conn.content)
        
        return raw_data
    
    def build_data(self, raw_data):
        if self.verbose: self._verbose('build_data', 'Building data frame...')
            
        if self.active_pull in ('asteroid_belts', 'moons'):
            planet_ids = pd.DataFrame(raw_data, columns=['planet_id', 'id_val']).set_index('id_val')['planet_id']
            raw_data = [data[1] for data in raw_data]
        
        ids_need_data = self.get_updates(raw_data)
        
        data_frame = self.get_id_data(ids_need_data)
        data_frame = pd.DataFrame([self.parse_id_data(id_data) for id_data in data_frame])
        data_frame['record_time'] = self.timestamp
        
        if self.active_pull in ('asteroid_belts', 'moons'):
            data_frame['planet_id'] = data_frame[self.settings['id_params'][self.active_pull]['id_col']].map(planet_ids)
        
        if self.verbose: self._verbose('build_data', 'Data frame built.')
            
        return data_frame
    
    def get_updates(self, raw_data):
        col_name = self.settings['id_params'][self.active_pull]['id_col']
        self.active_ids[self.active_pull] = pd.DataFrame(raw_data, columns=[col_name])[col_name]
        current_ids = self.get_current_ids()
        ids_need_data = self.active_ids[self.active_pull][~self.active_ids[self.active_pull].isin(current_ids)].reset_index(drop=True)
        return ids_need_data
    
    def get_current_ids(self):
        self.connect_maria()
        current_ids = pd.read_sql(
            self.sql['current_ids'].format(**self.settings['id_params'][self.active_pull], table=self.script_vals['table'][self.active_pull]),
            self.conn['maria']
        )[self.settings['id_params'][self.active_pull]['id_col']]
        self.conn['maria'].close()
        return current_ids
    
    def get_id_data(self, ids_need_data):
        if self.verbose: self._verbose('get_id_data', 'Getting structure name data...')
        
        if self.auth:
            self.headers['Authorization'] = 'Bearer {token}'.format(**{
                'token': self.auth_token
            })
            
        pool = Pool(self.thread_max)
        id_data = pool.map(self.grab_id_data, ids_need_data)

        if self.verbose: self._verbose('get_id_data', 'Raw structure name acquired.')
        
        return id_data
    
    def grab_id_data(self, id_val):
        url_params = {
            **self.url_params,
            'id_val': id_val,
            'pull_name': self.active_pull
        }
        data_conn = self.url_get(
            self.url['id_data'].format(**url_params),
            self.headers,
            self.params
        )
        return (id_val, data_conn.content)
    
    def parse_id_data(self, id_data):
        id_val, id_data = id_data
        try:
            id_data = js.loads(id_data)
        except:
            id_data = json.loads(id_data)
        
        id_item = {
            self.settings['id_params'][self.active_pull]['id_col']: id_val
        }
        
        for key, path in self.mappings[self.active_pull].items():
            id_item[key] = self.parse_dict(id_data, path)
        
        for sub_key in self.settings['id_params'][self.active_pull]['sub_keys']:
            if sub_key not in self.buffers.keys(): self.buffers[sub_key] = []
            buffer_item = self.parse_dict(id_data, sub_key)
            if buffer_item is None: continue
                
            if sub_key == 'planets':
                if 'asteroid_belts' not in self.buffers.keys(): self.buffers['asteroid_belts'] = []
                if 'moons' not in self.buffers.keys(): self.buffers['moons'] = []
                for planet_system in buffer_item:
                    self.buffers[sub_key].append(planet_system['planet_id'])
                    self.buffers['asteroid_belts'].extend([(planet_system['planet_id'], belt_id) for belt_id in planet_system.get('asteroid_belts', [])])
                    self.buffers['moons'].extend([(planet_system['planet_id'], moon_id) for moon_id in planet_system.get('moons', [])])
            else:
                self.buffers[sub_key].extend(buffer_item)
        
        return id_item
    
    def parse_dict(self, val, path):
        for path_item in path.split('.'):
            val = val.get(path_item, None)
            if val is None: break
        return val
        
    def data_inserter(self, data_frames):
        if self.verbose: self._verbose('insert_data', 'Inserting records...')
            
        cur = self.conn['maria'].cursor()
            
        for key, data_frame in data_frames.items():
            self.active_pull = key
            
            insert_script = self.sql['insert'].format(**{
                'table': self.script_vals['table'][key],
                'cols': ','.join(data_frame.columns),
                'vals': ','.join(['%s']*data_frame.columns.size),
                'upsert': ','.join(['{col}=VALUES({col})'.format(col=col) for col in data_frame.columns])
            })
        
            for row in data_frame.itertuples():
                try:
                    row = [val if pd.notnull(val) else None for val in row[1:]]
                    cur.execute(insert_script, row)
                except:
                    print(row)
                    print(insert_script)
                    raise Exception
            self.conn['maria'].commit()
        
            if self.delete: self.data_deleter(cur)
            
        cur.close()
        
        if self.verbose: self._verbose('insert_data', 'Records inserted.')
    
    def data_deleter(self, cur):
        if self.verbose: self._verbose('data_deleter', 'Deleting old records...')
        delete_script = self.sql['delete'].format(
            table=self.script_vals['table'][self.active_pull],
            id_col=self.settings['id_params'][self.active_pull]['id_col'],
            id_vals=','.join(self.active_ids[self.active_pull].astype(str))
        )
        cur.execute(delete_script)
        self.conn['maria'].commit()

In [3]:
api = ApiRegions()
api.run_process()

KeyboardInterrupt: 

In [ ]:
api.buffers['planets']